In [ ]:

import os
import re
import docker
from datetime import datetime, timezone, timedelta

# UTC → IST converter
def utc_to_ist(dt: datetime) -> datetime:
    ist_offset = timedelta(hours=5, minutes=30)
    return dt + ist_offset

def extract_datetime_from_logs(logs: str) -> datetime | None:
    # Pattern 1: "25-08-2025 16:12:37.929584930"
    match1 = re.search(r"(\d{2}-\d{2}-\d{4} \d{2}:\d{2}:\d{2}(?:\.\d{1,9})?)", logs)
    if match1:
        ts_str = match1.group(1)
        if "." in ts_str:
            # split seconds and fractional part
            base, frac = ts_str.split(".")
            # keep only first 6 digits for microseconds
            frac = (frac + "000000")[:6]
            ts_str = f"{base}.{frac}"
            return datetime.strptime(ts_str, "%d-%m-%Y %H:%M:%S.%f").replace(tzinfo=timezone.utc)
        else:
            return datetime.strptime(ts_str, "%d-%m-%Y %H:%M:%S").replace(tzinfo=timezone.utc)

    # Pattern 2: "Mon, 25 Aug 2025 15:40:16.123456 UTC"
    match2 = re.search(r"([A-Za-z]{3}, \d{2} [A-Za-z]{3} \d{4} \d{2}:\d{2}:\d{2}(?:\.\d{1,9})?) UTC", logs)
    if match2:
        ts_str = match2.group(1)
        if "." in ts_str:
            base, frac = ts_str.split(".")
            frac = (frac + "000000")[:6]
            ts_str = f"{base}.{frac}"
            return datetime.strptime(ts_str, "%a, %d %b %Y %H:%M:%S.%f").replace(tzinfo=timezone.utc)
        else:
            return datetime.strptime(ts_str, "%a, %d %b %Y %H:%M:%S").replace(tzinfo=timezone.utc)

    # Pattern 3: "Current UTC time: 2025-08-25 15:40:17.811212832 UTC"
    match3 = re.search(r"Current UTC time: (\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}(?:\.\d{1,9})?)", logs)
    if match3:
        ts_str = match3.group(1)
        if "." in ts_str:
            base, frac = ts_str.split(".")
            frac = (frac + "000000")[:6]
            ts_str = f"{base}.{frac}"
            return datetime.strptime(ts_str, "%Y-%m-%d %H:%M:%S.%f").replace(tzinfo=timezone.utc)
        else:
            return datetime.strptime(ts_str, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc)

    return None


def create_container(image_name):
    client = docker.from_env()
    container = client.containers.run(image_name, detach=True)
    return container

def main():
    images = [
        "yaskam24/c:factors", "yaskam24/c:sine", "yaskam24/c:cosine", "yaskam24/c:factorial",
        "yaskam24/go:factors", "yaskam24/go:sine", "yaskam24/go:cosine", "yaskam24/go:factorial",
        "yaskam24/rust:factors", "yaskam24/rust:sine", "yaskam24/rust:cosine", "yaskam24/rust:factorial"
    ]

    for image in images:
        cont = create_container(image)
        result = cont.wait(timeout=10)
        logs = cont.logs().decode()

        print(f"\nImage: {image}")
        print(f"Container ID: {cont.id}")
        print("Exit code:", result["StatusCode"])
        print("Logs:\n", logs)

        # Extract datetime
        dt = extract_datetime_from_logs(logs)
        if dt:
            ist_time = utc_to_ist(dt)
            print(f"Extracted UTC time: {dt}")
            print(f"Converted IST time: {ist_time}")
        else:
            print("⚠️ No datetime found in logs.")

        # Save results
        with open("benchmark_results.txt", "a") as f:
            f.write(f"Image: {image}\n")
            f.write(f"Container ID: {cont.id}\n")
            f.write(f"Exit code: {result['StatusCode']}\n")
            f.write(f"Logs:\n{logs}\n")
            if dt:
                f.write(f"UTC: {dt}, IST: {ist_time}\n")
            f.write("\n")

if __name__ == "__main__":
    main()


In [ ]:
import subprocess
dockerfile_path = [
    "/home/yash/test/benchmark/Math/go/cosine/Dockerfile",
    "/home/yash/test/benchmark/Math/go/sine/Dockerfile",
    "/home/yash/test/benchmark/Math/go/factorial/Dockerfile",
    "/home/yash/test/benchmark/Math/go/factors/Dockerfile",
    "/home/yash/test/benchmark/Math/rust/cosine/Dockerfile",
    "/home/yash/test/benchmark/Math/rust/sine/Dockerfile",
    "/home/yash/test/benchmark/Math/rust/factorial/Dockerfile",
    "/home/yash/test/benchmark/Math/rust/factors/Dockerfile",
    "/home/yash/test/benchmark/Math/c/cosine/Dockerfile",
    "/home/yash/test/benchmark/Math/c/sine/Dockerfile",
    "/home/yash/test/benchmark/Math/c/factorial/Dockerfile",
    "/home/yash/test/benchmark/Math/c/factors/Dockerfile",
    "/home/yash/test/benchmark/Math/c/factors/Dockerfile.alpine",
    "/home/yash/test/benchmark/Math/c/factorial/Dockerfile.alpine",
    "/home/yash/test/benchmark/Math/c/sine/Dockerfile.alpine",
    "/home/yash/test/benchmark/Math/c/cosine/Dockerfile.alpine",
    "/home/yash/test/benchmark/Math/go/cosine/Dockerfile.alpine",
    "/home/yash/test/benchmark/Math/go/sine/Dockerfile.alpine",
    "/home/yash/test/benchmark/Math/go/factorial/Dockerfile.alpine",
    "/home/yash/test/benchmark/Math/go/factors/Dockerfile.alpine",
    "/home/yash/test/benchmark/Math/rust/cosine/Dockerfile.alpine",
    "/home/yash/test/benchmark/Math/rust/sine/Dockerfile.alpine",
    "/home/yash/test/benchmark/Math/rust/factorial/Dockerfile.alpine",
    "/home/yash/test/benchmark/Math/rust/factors/Dockerfile.alpine",
]

for i in dockerfile_path:
    path = i.split("/")
    context = path[:-2]
    context_path = "/".join(context)
    print(context_path)
    result = subprocess.run(["docker", "build", "-t", f"yaskam24/{path[-3]}:{path[-2]}", "-f", i, context_path], check=True)
    print(result)

/home/yash/test/benchmark/Math/go
/home/yash/test/benchmark/Math/go
/home/yash/test/benchmark/Math/go
/home/yash/test/benchmark/Math/go
/home/yash/test/benchmark/Math/rust
/home/yash/test/benchmark/Math/rust
/home/yash/test/benchmark/Math/rust
/home/yash/test/benchmark/Math/rust
/home/yash/test/benchmark/Math/c
/home/yash/test/benchmark/Math/c
/home/yash/test/benchmark/Math/c
/home/yash/test/benchmark/Math/c
/home/yash/test/benchmark/Math/c
/home/yash/test/benchmark/Math/c
/home/yash/test/benchmark/Math/c
/home/yash/test/benchmark/Math/c
/home/yash/test/benchmark/Math/go
/home/yash/test/benchmark/Math/go
/home/yash/test/benchmark/Math/go
/home/yash/test/benchmark/Math/go
/home/yash/test/benchmark/Math/rust
/home/yash/test/benchmark/Math/rust
/home/yash/test/benchmark/Math/rust
/home/yash/test/benchmark/Math/rust
